# Gender

## Queries: case count

`MATCH (n:Case) RETURN COUNT(n)`

13,724,299 total cases

`MATCH (n:Case {female_defendant: "0 male"}) RETURN COUNT(n)`

5,150,275 cases with male defendants (0.429965348313 of total classified)

`MATCH (n:Case {female_defendant: "1 female"}) RETURN COUNT(n)`

6,828,074 cases with female defendants (0.570034651687 of total classified)

1,745,950 cases with unclassified gender defendant

**Summary:** Slight majority of cases have female defendants (unclear which way nonclassified case defendants lean).

## Queries: judge count

`MATCH (n:Judge) RETURN count(n)`

98,478 total judges

`MATCH (n:Judge {judge_female: "0 nonfemale"}) RETURN count(n)`

67,540 nonfemale judges (0.712883409681 of total)

`MATCH (n:Judge {judge_female: "1 female"}) RETURN count(n)`

27,202 female judges (0.287116590319 of total)

**Summary:** Large majority of judges are nonfemale.

## Query: centraility

`MATCH (n:Case) RETURN n.case_id AS id, n.female_defendant, n.page_rank_score ORDER BY n.page_rank_score DESC`

Of top 25 most central cases, 9 defendants are female, 3 are male (rest are undefined); females are spots 2-4 (1 is undefined). The most central three cases with female defendants all were in the state Assam (id: 23) and used the Code of Criminal Procedure.

`MATCH (n:Case) RETURN AVG(n.page_rank_score)`

average page rank score for cases defendants: 0.5268351389034323

`MATCH (n:Case) WHERE n.female_defendant = "1 female" RETURN AVG(n.page_rank_score)`

average page rank score for cases with female defendants: 0.5205913061322096

`MATCH (n:Case) WHERE n.female_defendant = "0 nonfemale" RETURN AVG(n.page_rank_score)`

average page rank score for cases with male defendants: 0.5260501571201543

`MATCH (n:Judge) RETURN AVG(n.page_rank_score)`

average page rank score for judges: 22.285623828167502

`MATCH (n:Judge) WHERE n.judge_female = "1 female" RETURN AVG(n.page_rank_score)`

average page rank score for female judges: 19.653472457940392

`MATCH (n:Judge) WHERE n.judge_female = "0 nonfemale" RETURN AVG(n.page_rank_score)`

average page rank score for non-female judges: 23.559010410462946

## Query: exploring relations between female defendants and female judges

Key:
- Female - Female : female defendant, female judge
- Female - Nonfemale : female defendant, nonfemale judge
- Male - Female : male defendant, female judge
- Male - Nonfemale : male defendant, nonfemale judge

`MATCH (n:Case)-[r:JUDGED]-(m:Judge) RETURN n.female_defendant AS defendant_gender, m.judge_female AS judge_gender, COUNT(r) AS frequency, AVG(n.page_rank_score) AS average_page_rank`

- female - female,	count: 244390,	average page rank: 0.6450088000710372	
- female - nonfemale,	count: 635649,average page rank: 0.6524839655877823
- male - female,	count: 259412,	average page rank: 0.651472444456763
- male - nonfemale,	count: 740082,	average page rank: 0.6536335921621823

### Proportions
- Female - Female: 0.13002698
- Female - Nonfemale: 0.338195179
- Male - Female: 0.138019391
- Male - Nonfemale: 0.39375845

## Dispositons

### "accepted" disposition (disposition code 4)
- Female - Female 0.084631674
- Female - Nonfemale 0.304719407
- Male - Female 0.090001513
- Male - Nonfemale 0.520647406
- Female judges are under-represented in accepted dispositions.

### "acquitted" disposition (disposition code 5)
- Female - Female 0.199222829
- Female - Nonfemale 0.389590135
- Male - Female 0.141371152
- Male - Nonfemale 0.269815884
- Female judges with female defendants are slightly overrepresented, while male judges with male defendants are underrepresented.

### "allowed" disposition (disposition code 6)
- Female - Female 0.10209578
- Female - Nonfemale 0.411833312
- Male - Female 0.107095634
- Male - Nonfemale 0.378975274
- Female defendants with male judges are underrepresented.

### "bail granted" disposition (disposition code 8)
- Female - Female 0.21611143
- Female - Nonfemale 0.297960942
- Male - Female 0.205341758
- Male - Nonfemale 0.28058587
- Female judges are overrepresented, while male judges are underrepresented.

### "bail refused" disposition (disposition code 9)
- Female - Female 0.093635995
- Female - Nonfemale 0.446291042
- Male - Female 0.119983786
- Male - Nonfemale 0.446291042
- Male judges are overrepresented, while female judges are underrepresented.

### "committed" disposition (disposition code 12)
- Female - Female 0.140399988
- Female - Nonfemale 0.390221144
- Male - Female 0.140399988
- Male - Nonfemale 0.275636839
- Male judges with male defendants are underrepresented.

### "convicted" disposition (disposition code 17)
- Female - Female 0.160742343
- Female - Nonfemale 0.258043949
- Male - Female 0.214140954
- Male - Nonfemale 0.367072754

### "decided" disposition (disposition code 18)
- Female - Female 0.07097254
- Female - Nonfemale 0.226865455
- Male - Female 0.121231908
- Male - Nonfemale 0.580930098
- Male judges with male defendants are overrepresnted.

### "dismissed" disposition (disposition code 20)
- Female - Female 0.127693564
- Female - Nonfemale 0.368198525
- Male - Female 0.122949047
- Male - Nonfemale 0.381158863

### "disposed" disposition (disposition code 22)
- Female - Female 0.096732335
- Female - Nonfemale 0.279913121
- Male - Female 0.172709546
- Male - Nonfemale 0.450644998
- Female defendants are underrepresented while male defendants are overrepresented.

### "fine" disposition (disposition code 26)
- Female - Female 0.07864667
- Female - Nonfemale 0.276319705
- Male - Female 0.143129203
- Male - Nonfemale 0.501904422
- Female judges with female defendants are underrepresented, while male judges with male defendants are overrepresented.

### "judgement" disposition (disposition code 27)
- Female - Female 0.196561157
- Female - Nonfemale 0.406747427
- Male - Female 0.10855803
- Male - Nonfemale 0.288133385
- Male defendants are underrepresented, while female defendants are overrepresented.

### "judgement" disposition (disposition code 27)
- Female - Female 0.196561157
- Female - Nonfemale 0.406747427
- Male - Female 0.10855803
- Male - Nonfemale 0.288133385
- Male defendants are underrepresented, while female defendants are overrepresented.

### "referred to lok adalat" disposition (disposition code 37)
- Female - Female 0.109843334
- Female - Nonfemale 0.216435117
- Male - Female 0.148211646
- Male - Nonfemale 0.525509902
- Male defendants are overrepresented, while female defendants are underrepresented.

### "reject" disposition (disposition code 38)
- Female - Female 0.083542211
- Female - Nonfemale 0.452882513
- Male - Female 0.075355981
- Male - Nonfemale 0.388219295
- Female judges are underrepresented.

### "reject" disposition (disposition code 43)
- Female - Female 0.108259147
- Female - Nonfemale 0.322216699
- Male - Female 0.149366593
- Male - Nonfemale 0.420157561

### "withdrawn" disposition (disposition code 45)
- Female - Female 0.137989432
- Female - Nonfemale 0.324007587
- Male - Female 0.164340875
- Male - Nonfemale 0.373662105

## Query: exploring relations between cases and acts

`MATCH (n:Case)-[r:USES_ACT]-(m:Act) RETURN n.female_defendant AS defendant_gender, m.act_sum AS act, COUNT(r) AS frequency ORDER BY m.act_sum`

## Query: exploring relations disposition

`MATCH (n:Case)-[r:JUDGED]-(m:Judge) RETURN n.disp_name as disp, n.female_defendant AS defendant_gender, m.judge_female as judge_gender, COUNT(n) ORDER BY n.disp_name`